# Lux AI Season 1 Python Tutorial Notebook

Welcome to Lux AI Season 1!

This notebook is the basic setup to use Jupyter Notebooks and the `kaggle-environments` package to develop your bot. If you plan to not use Jupyter Notebooks or any other programming language, please see our [Github](https://github.com/Lux-AI-Challenge/Lux-Design-2021). The following are some important links!

- Competition Page: https://www.kaggle.com/c/lux-ai-2021/

- Online Visualizer: https://2021vis.lux-ai.org/

- Specifications: https://www.lux-ai.org/specs-2021

- Github: https://github.com/Lux-AI-Challenge/Lux-Design-2021

- Bot API: https://github.com/Lux-AI-Challenge/Lux-Design-2021/tree/master/kits

And if you haven't done so already, we **highly recommend** you join our Discord server at https://discord.gg/aWJt3UAcgn or at the minimum follow the kaggle forums at https://www.kaggle.com/c/lux-ai-2021/discussion. We post important announcements there such as changes to rules, events, and opportunities from our sponsors!

Now let's get started!

## Prerequisites

We assume that you have a basic knowledge of Python and programming. It's okay if you don't know the game specifications yet! Feel free to always refer back to https://www.lux-ai.org/specs-2021.

## Basic Setup

First thing to verify is that you have **Node.js v12 or above**. The engine for the competition runs on Node.js (for many good reasons including an awesome visualizer) and thus it is required. You can download it [here](https://nodejs.org/en/download/). You can then verify you have the appropriate version by running


Next, we have to import the `make` function from the `kaggle_environments` package

In [1]:
from kaggle_environments import make

Loading environment football failed: No module named 'gfootball'


The `make` function is used to create environments that can then run the game given agents. Agents refer to programmed bots that play the game given observations of the game itself. 

In addition to making the environment, you may also pass in special configurations such as the number of episode steps (capped at 361) and the seed.

Now lets create our environment using `make` and watch a Episode! (We will be using the seed 562124210 because it's fun)

In [2]:
# Devastator #1 agent

In [22]:
# Completely new agent type from scratch. Seems promising and fun! We're not begrudged by the starter kit's doodoo logic (although the structure is still there).
# We have clear ways to improve this. See sticky notes.
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_objects import Unit
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys


### Define helper functions
# Fix using A* so it works with this too


# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resource(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile

def find_closest_city_tile(pos, player):
    """
    pos: Position Object
    player: Player object


    Returns 'closest_city_tile': CityTile Object 
    """
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile



############## END OF HELPER FUNCTIONS ###############
######################################################
######################################################
######################################################
######################################################



#### Agent 3 - Devastator (pre-taking their code) 
#Aim of the game: Fucking duplicate as fast as you can. Duplicate and eat the world.

game_state = None
def dev1(observation, configuration):
    ########### Code / Bot starts here ############
    ###############################################

    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])

    actions = []
    ## Helper functions
    ### VOID FUNCTIONS BELOW
    def researched(resource):
        """
        given a Resource object, return whether the player has researched the resource type
        """
        if resource.type == Constants.RESOURCE_TYPES.WOOD:
            return True
        if resource.type == Constants.RESOURCE_TYPES.COAL \
            and player.research_points >= GAME_CONSTANTS['PARAMETERS']['RESEARCH_REQUIREMENTS']['COAL']:
                return True
        if resource.type == Constants.RESOURCE_TYPES.URANIUM \
            and player.research_points >= GAME_CONSTANTS['PARAMETERS']['RESEARCH_REQUIREMENTS']['URANIUM']:
                return True
        return False

    def get_cells(cell_type):  # resource, researched resource, player citytile, enemy citytile, empty
        """
        Given a cell type, returns a list of Cell objects of the given type
        Options are: ['resource', 'researched resource', 'player citytile', 'enemy citytile', 'empty']
        """
        cells_of_type = []
        for y in range(height):
            for x in range(width):
                cell = game_state.map.get_cell(x, y)
                if (
                       ( cell_type == 'resource' and cell.has_resource() ) \
                    or ( cell_type == 'researched resource' and cell.has_resource() and researched(cell.resource) ) \
                    or ( cell_type == 'player citytile' and cell.citytile is not None and cell.citytile.team == observation.player ) \
                    or ( cell_type == 'enemy citytile' and cell.citytile is not None and cell.citytile.team != observation.player ) \
                    or ( cell_type == 'empty' and cell.citytile is None and not cell.has_resource() )
                ):
                    cells_of_type.append(cell)

        return cells_of_type

    def find_nearest_position(target_position, option_positions):
        # Should we change to find nearest cell?
        """
        target_position: Position object
        option_positions: list of Position, Cell, or Unit objects (must all be the same type)
        finds the closest option_position to the target_position

        Returns: Closest_position -> Position object
        """

        # convert option_positions list to Position objects
        if type(option_positions[0]) in [Cell, Unit]:
            option_positions = [cell.pos for cell in option_positions]

        # find closest position
        closest_dist = math.inf
        closest_position = None
        for position in option_positions:
            dist = target_position.distance_to(position)
            if dist < closest_dist:
                closest_dist = dist
                closest_position = position

        return closest_position

    target_tiles = [] # to help avoid collisions. You'd want to put this somewhere inside the agent (so it can append target tiles over time and figure shit out)
    #That's our 2nd best alternative since we haven't done linear assignment problem thingy yet.
    def move_unit(unit, position):
        """
        moves the given unit towards the given position
        also checks basic collision detection, and adds annotations for any movement
        """

        direction = unit.pos.direction_to(position)
        target_tile = unit.pos.translate(direction, 1)

        # if target_tile is not being targeted already, move there
        if target_tile not in target_tiles or target_tile in [tile.pos for tile in citytile_cells]:
            target_tiles.append(target_tile)
            actions.append(unit.move(direction))
            actions.append(annotate.line(unit.pos.x, unit.pos.y, position.x, position.y))

        # else, if it is being targetted mark an X on the map. Need to fix this to handle collision
        else:
            actions.append(annotate.x(target_tile.x, target_tile.y))
#         print("YO BROOO", type(target_tile))


    def go_home(unit, citytile_cells):
        """
        moves the given unit towards the nearest citytile
        """

        nearest_citytile_position = find_nearest_position(unit.pos, citytile_cells)
        move_unit(unit, nearest_citytile_position)


    ###############################################
    #################### Our Code  ################
    ###############################################


    #### Fucking ALGO CODE
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    # add debug statements like so!
    if game_state.turn == 0:
        print("I am devastator agent!!", file=sys.stderr)
        trees = get_cells('researched resource')
        tree_pos = []
        for tree in trees:
            tree_pos.append(str(tree.pos))
        print(tree_pos, file=sys.stderr)
        trees_other = find_resources(game_state)
        print("other:", type(trees_other), [str(tree.pos) for tree in trees_other], file=sys.stderr)



    resource_tiles = find_resources(game_state)
    citytile_cells = get_cells('player citytile')
    print("Yo BROOO WASSUP", type(citytile_cells[0]), file=sys.stderr)
    num_citytiles = len(citytile_cells)
    is_night = game_state.turn % 40 > 30 # Bool - True if it's night time
#     num_units = len(player.units)
#     print("Num citytile or units", num_citytiles, num_units, file=sys.stderr)


    ### City actions / citybuilding (put this shit ijn functions later)
    """Priority list for cities:
    Worker
    Research"""
    for k, city in player.cities.items():
        ## Running through each citytile for logic
        for citytile in city.citytiles:
            if citytile.can_act():
                # Either make a new worker or just research.
                if len(citytile_cells) > len(player.units): 
                    action = citytile.build_worker()
                    actions.append(action)

                else:
                    action = citytile.research()
                    actions.append(action)

    ### Unit Actions ####

    for unit in player.units:
        """
        Priority list for units:
        Place a city if possible (should we put a weight on adjacent cities?)

        else;
            Mine
            If cargo full and you don't wanna place;
                Return to city

        """
        # TODO - We could have the bot find the closest empty cell to place a city too. Actually, let's do that now.
        closest_city_tile = find_closest_city_tile(unit.pos, player) 
        ## Code for Workers ##
        if unit.is_worker() and unit.can_act():

            # Check if it has no cargo space
            if unit.get_cargo_space_left() == 0 and unit.can_build(game_state.map): #I.e it's at 100 cargo space (the amt required to build a city):
                print("I'm gonna build a city because I have full inventory + I can build!", file=sys.stderr)
                actions.append(unit.build_city())

            # We're full but unbuildable rn, find the closest place to build to. THis might go horribly wroong lol
            elif unit.get_cargo_space_left() == 0:
                empty_cells = get_cells('empty')
                closest_empty_cell_pos = find_nearest_position(unit.pos, empty_cells)
                move_unit(unit, closest_empty_cell_pos)

            else: #I.e you might have 0 cargo but can't build
                # Mine if you have cargo space  
                # Else go home
                if unit.get_cargo_space_left() > 0:
                    closest_resource_tile = find_closest_resource(unit.pos, player, resource_tiles)
                    if closest_resource_tile is not None and not is_night:
                        # Move to closest resource if it's day time OR we don't have enough fuel
                        move_unit(unit, closest_resource_tile.pos)
                    else:
                        print("Well.. Shit. I'm unit,", unit.id ,"Apparently I can't find any resources. Or it's nighttime", file=sys.stderr)

                    # go_mine(unit, resources) -> Make this void function sometime? (i.e function that does stuff)

                else:
                    # If we have no cargo space & can't build.
                    go_home(unit, citytile_cells)
                #  Go home 


    print("Super secret. Turn:", game_state.turn, "Action list:", actions, file=sys.stderr)

    return actions

In [23]:
# # create the environment. You can also specify configurations for seed and loglevel as shown below. If not specified, a random seed is chosen. 
# # loglevel default is 0. 
# # 1 is for errors, 2 is for match warnings such as units colliding, invalid commands (recommended)
# # 3 for info level, and 4 for everything (not recommended)
# # set annotations True so annotation commands are drawn on visualizer
# # set debug to True so print statements get shown
# env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, debug=True)

In [24]:
# # run a match between two simple agents, which are the agents we will walk you through on how to build!
# steps = env.run(["simple_agent", "simple_agent"])
# # if you are viewing this outside of the interactive jupyter notebook / kaggle notebooks mode, this may look cutoff
# # render the game, feel free to change width and height to your liking. We recommend keeping them as large as possible for better quality.
# # you may also want to close the output of this render cell or else the notebook might get laggy
# env.render(mode="ipython", width=1200, height=800)

Ok so woah, what just happened? We just ran a match, that's what :)

There's a number of quality of life features in the visualizer, which you can also find embedded on the kaggle competition page when watching replays or on the online visualizer when using replay files. 

If you find this replay viewer slow, you can also download a local copy of this replay viewer in addition to lowering the graphics quality, see https://github.com/Lux-AI-Challenge/LuxViewer2021 for instructions.

At this point, we recommend reading the [game specifications](https://www.lux-ai.org/specs-2021) a bit more to understand how to build a bot that tries to win the game.

## Helper functions

In [25]:
# ### Define helper functions

# # this snippet finds all resources stored on the map and puts them into a list so we can search over them
# def find_resources(game_state):
#     resource_tiles: list[Cell] = []
#     width, height = game_state.map_width, game_state.map_height
#     for y in range(height):
#         for x in range(width):
#             cell = game_state.map.get_cell(x, y)
#             if cell.has_resource():
#                 resource_tiles.append(cell)
#     return resource_tiles

# # the next snippet finds the closest resources that we can mine given position on a map
# def find_closest_resources(pos, player, resource_tiles):
#     closest_dist = math.inf
#     closest_resource_tile = None
#     for resource_tile in resource_tiles:
#         # we skip over resources that we can't mine due to not having researched them
#         if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
#         if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
#         dist = resource_tile.pos.distance_to(pos)
#         if dist < closest_dist:
#             closest_dist = dist
#             closest_resource_tile = resource_tile
#     return closest_resource_tile

# def find_closest_city_tile(pos, player):
#     """
#     pos: Position Object
#     player: Player object


#     Returns 'closest_city_tile': CityTile Object 
#     """
#     closest_city_tile = None
#     if len(player.cities) > 0:
#         closest_dist = math.inf
#         # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
#         # contains the information of all citytiles in that city
#         for k, city in player.cities.items():
#             for city_tile in city.citytiles:
#                 dist = city_tile.pos.distance_to(pos)
#                 if dist < closest_dist:
#                     closest_dist = dist
#                     closest_city_tile = city_tile
#     return closest_city_tile


In [26]:
# # lets look at some of the resources found
# resource_tiles = find_resources(game_state)
# cell = resource_tiles[0]
# print(game_state.turn)
# print("Cell at", cell.pos, "has")
# print(cell.resource.type, cell.resource.amount)
# # for cell in resource_tiles:
# #     print(cell.pos)

In [27]:
# # lets see if we do find some close resources
# cell = find_closest_resources(Position(1, 1), game_state.players[0], resource_tiles)
# print("Closest resource at", cell.pos, "has")
# print(cell.resource.type, cell.resource.amount)

Yoinked from helper function code notebooks from Kaggle:
## Void Functions / Returns
(Functions that *do* something and don't return anything (basically they're just procedures or whatever))

With this function, we are ready to start surviving the nights! The code below rewrites our agent to now have units who have full cargos to head towards the closest citytile and drop off their resources to fuel the city.


# Agent Creation!

In [28]:
# Agent 2 (this is like the 'develop' branch of our Agent 1 - solid bot (see the simple file)) below:
# I think we just work on agents in vsc, tourney mode run them and then bring them here if we want to debug.

In [29]:
"""
Started 29/8/21. We will try to push this bot to 1100. If not, we're gonna push this bot to eat everything on the map a high % of the time.

Planetary Devastation - Instead of eaching all of a cluster first then moving on, *spread your cancer* (i.e send 1 worker) to a different cluster and make them do their own devastation. So we just devastate  the world as fast as we can. ---> Floodfill.

Maybe figure out better boilerplate?
https://www.kaggle.com/superant/halite-boilerbot
"""
from lux import game
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys

from scipy.ndimage import measurements
import numpy as np
"""
Planetary Devastation, Funny Agent:
Destroy the environment as fast as possible...

> Collect resources
> Build city ASAP
> When you have enough workers, identify resource clusters. Send workers to different resource clusters.
---> Make sure they can live by the time they get there?

"""


"""Cluster identification:
First. 
Get researched resource cells.

Loop through them -> Check if 

I want a dictionary:
Trees: {'1': [(cluster1 trees so Tree1), (Tree2), ...], '2': (cluster2 trees so Tree1, Tree2,...), 'n': ...} # Dictionary of trees and their clusters.

for tree in trees (get_cell'd):
	if tree.pos.adjacent_to							

/// After identifyng:
Then, I  basically want to assign an agent to move to a cluster '1', or '2', etc.


Check notebook!!! "# Then, for each agent, we want to find the closest non_zero cluster THAT HASN'T BEEN PICKED YET. HOLY SHIT THAT'S IT. THIS IS THE MOST IMPORTANT FUCKING LINE IN THIS WHOLE NTOEBOOK
""

"""
	
def manhattan_distance(x1, x2, y1, y2):
	return abs(x1-x2) + abs(y1-y2)

class ClusterTracker():
	"""
	Anything to do with cluster creation / tracking / identification 
	"""
	def __init__(self, game_state):
		# Initialise 0 matrix to width, height of map.
		width, height = game_state.map_width, game_state.map_height
		self.empty_matrix = np.zeros([width, height])

		# Initialise clsuter matrices for each different resource.
		self.tree_matrix = self.empty_matrix
		self.coal_matrix = self.empty_matrix
		self.uranium_matrix = self.empty_matrix
		
		
	def create_cluster_matrix(self, game_state, resource_type):
		""" Creates tree matrix 
		Updates the tree matrix aswell btw.
		Args:
		game_state -> game_state
		resource_type: "wood", "coal", "uranium"
		Returns: Tree matrix
		Example:
		[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
		[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
		[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
		[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
		[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
		Note that to get a (x,y) value of a matrix you must go matrix[y][x]"""

		# Resets cluster matrix based on what resource we're getting so old resources don't stay in there.
		resource_matrix = self.empty_matrix
		if resource_type == 'wood':
			self.tree_matrix = self.empty_matrix 
		elif resource_type == 'coal':
			self.coal_matrix = self.empty_matrix 
		else:
			self.uranium_matrix = self.empty_matrix 

		## The stuff.
		width, height = game_state.map_width, game_state.map_height

		for y in range(height):
			for x in range(width):
				cell = game_state.map.get_cell(x, y)
				# Debugging, just to see what resources it 'sees'.
				# try:
				# 	print(cell.resource.type)
				# except:
				# 	pass
				# Sets any position where there's a tree to 1 in the tree_matrix
				if resource_type == 'wood' and cell.has_resource() and cell.resource.type == Constants.RESOURCE_TYPES.WOOD: # If it's a tree add it to the matrix.
					self.tree_matrix[y][x] = 1
					resource_matrix[y][x] = 1 
				# The =1 / 2 / 3 is arbitrary. It's just so we can differentiate better if we have to debug.
				elif resource_type == 'coal' and cell.has_resource() and cell.resource.type == Constants.RESOURCE_TYPES.COAL: # If it's a coal add it to the matrix.
					self.coal_matrix[y][x] = 2
					resource_matrix[y][x] = 2 
				elif resource_type == 'uranium' and cell.has_resource() and cell.resource.type == Constants.RESOURCE_TYPES.URANIUM: # If it's a coal add it to the self.coal_matrix.
					self.uranium_matrix[y][x] = 3
					resource_matrix[y][x] = 3
					
		# Return cluster matrix of resource we were looking for.
		return resource_matrix
	
	def convert_to_cluster(self, resource_matrix):
		"""Get cluster information -> cluster matrix and position of the clustered resources
		 Resource matrix: The returned matrix of create_cluster_matrix. This is a matrix of the resources in the map
		 """
		cluster_matrix, num_clusters = measurements.label(resource_matrix) 
		""" Cluster matrix looks like
		 [[ 0  1  0  0  0  0  0  0  0  0  0  0  0  0  2  0]
		[ 3  0  4  0  0  0  0  0  0  0  0  0  0  5  0  6]
		[ 0  7  0  0  0  0  0  0  0  0  0  0  0  0  8  0]
		[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
		[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
		[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
		[ 0  0  0  9  0 10  0  0  0  0  0  0  0  0  0  0]
		[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
		[ 0  0 11 11 11  0  0  0  0  0  0  0  0  0  0  0]
		[ 0  0 11 11  0  0  0  0  0  0  0  0  0  0  0  0]
		[ 0 12  0 11 11  0  0  0  0  0  0  0  0  0  0  0]
		[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
		[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
		[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
		[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
		[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]"""
		# Cluster_matrix = matrix of all clusters IDENTIFIED (i.e each individual cluster is called '1', '2', etc... Like, each tree in cluster 1 is called '1' in the matrix.)
		# num_clusters is the number of clusters lol (for this resource).
		# See the jupyter notebook for cluster info.

		resource_positions = cluster_matrix.nonzero() # Gets a list of [[y],[x]] coordinates of the positions of trees, etc.
		return cluster_matrix, resource_positions


	def closest_cluster(self, unit, resource_positions, cluster_matrix):
		"""Returns the number of the closest non-zero cluster and its position as a TUPLE
		so
		Args:
		Unit
		non_zero_pos - A list of 2 arrays [[y], [x]] coordinates of the positions of clusters 
		matrix - Matrix of unique cluster locations
		Returns: [11, Pos(10, 9)] or some shit -> cluster_id 11 at Position Object (10, 9)"""
		start = unit.pos
		
		y_list = resource_positions[0] # A list
		x_list= resource_positions[1] # A list
		
		closest_dist = math.inf
		for i in range(len(y_list)): # Both x,y same length:
			y= y_list[i]
			x = x_list[i]
		
			dist = manhattan_distance(start.x, x, start.y, y)
			# dist = manhattan_distance(10, x, 2, y) # test
			
			if dist < closest_dist:
				closest_dist = dist
				closest_cluster_id = cluster_matrix[y][x]
				# So we can return it later for other functions
				bestx = x
				besty = y
		
		position = Position(bestx,besty) # Transform it into luxai's Position object just so it's more compatible and thematic.
				
		return [closest_cluster_id, position]  # E.g [4, (3,10)] -> Unique cluster 4 @ Position (3,10).


class Agent():
	def __init__(self):
		self.memory={}
		# self.trees = .. etc so OP OP and you can access/update them only when required. I mean this is more optimisation stuff ut stilkl cool.
		# Trackers

	def process_observation(self, observation):
		# updates gamestate and initializes at step zero
		if observation["step"] == 0:
			self.game_state = Game()
			self.game_state._initialize(observation["updates"])
			self.game_state._update(observation["updates"][2:])
			self.game_state.id = observation.player
		else:
			self.game_state._update(observation["updates"])
		return self.game_state



	# Now we just go to it (if another agent hasn't called dibs on it already).
	def move_to_cluster(unit, closest_cluster):
		"""
		Args:
		Unit - Unit object
		closest_cluster: List of cluster ID and closest tuple to a worker, as [11, (10,9)] example.
		
		Returns: NA, moves unit to fkin closest resource in that cluster ID"""
		cluster_id, cluster_resource_pos = closest_cluster
		move_unit(unit, cluster_resource_pos) # Might have to change move unit so we can accept tuple inputs / change cluster-pos to a Pos object.
		

	def __call__(self, observation, configuration):
		# What happens every time the agent is called.
		game_state = self.process_observation(observation)

		actions = []
		player = game_state.players[observation.player]
		opponent = game_state.players[(observation.player + 1) % 2]
		width, height = game_state.map.width, game_state.map.height

		## Clusters
		cluster_tracker = ClusterTracker(game_state)
		tree_matrix = cluster_tracker.create_cluster_matrix(game_state, "wood") # Don't use this anywhere but convert_to_cluster. In fact, this should probably just be an inside step for convert to cluster lol.
		tree_cluster_matrix, tree_positions = cluster_tracker.convert_to_cluster(tree_matrix) # Tree cluster matrix is a relabled tree matrix where each cluster has its own unique id (e.g all trees in cluster 1 relabled to '1'.)


	# add debug statements like so!
		if game_state.turn == 0:
			print("{Planetary Devastation is running!", file=sys.stderr)
			actions.append(annotate.circle(0, 0))


		for unit in player.units:
			cluster_id, cluster_pos = cluster_tracker.closest_cluster(unit, tree_positions, tree_cluster_matrix) # [11, (2,3)]
			# tree_cluster_matrix[unit.pos.y][unit.pos.x] = 69
			# print(tree_cluster_matrix, cluster_id, file=sys.stderr)
			print(type(cluster_pos))
			print("Closest cluster id is: {} at position {} at turn {}".format(cluster_id, cluster_pos, game_state.turn), file=sys.stderr)
			actions.append(annotate.line(unit.pos.x, unit.pos.y, cluster_pos.x, cluster_pos.y))

		return actions
	


# the rest only to make it work for Kaggle submission
agent = Agent()
def call_agent(obs, conf):
	return agent(obs,conf)

In [30]:
print('ga')
env = make("lux_ai_2021", configuration={"seed": 854150936, "loglevel": 2, "annotations": True}, debug=True)
# steps = env.run([call_agent, dev1])
steps = env.run([call_agent, dev1])

ga
<class 'lux.game_map.Position'>
{Planetary Devastation is running!
Closest cluster id is: 9 at position (4, 6) at turn 0
I am devastator agent!!
['(1, 0)', '(14, 0)', '(0, 1)', '(2, 1)', '(13, 1)', '(15, 1)', '(1, 2)', '(14, 2)', '(3, 6)', '(4, 6)', '(5, 6)', '(10, 6)', '(11, 6)', '(12, 6)', '(2, 8)', '(3, 8)', '(4, 8)', '(11, 8)', '(12, 8)', '(13, 8)', '(2, 9)', '(3, 9)', '(12, 9)', '(13, 9)', '(1, 10)', '(3, 10)', '(4, 10)', '(11, 10)', '(12, 10)', '(14, 10)']
other: <class 'list'> ['(1, 0)', '(7, 0)', '(8, 0)', '(14, 0)', '(0, 1)', '(2, 1)', '(13, 1)', '(15, 1)', '(1, 2)', '(14, 2)', '(3, 6)', '(4, 6)', '(5, 6)', '(10, 6)', '(11, 6)', '(12, 6)', '(2, 8)', '(3, 8)', '(4, 8)', '(11, 8)', '(12, 8)', '(13, 8)', '(2, 9)', '(3, 9)', '(12, 9)', '(13, 9)', '(1, 10)', '(3, 10)', '(4, 10)', '(11, 10)', '(12, 10)', '(14, 10)', '(5, 15)', '(6, 15)', '(9, 15)', '(10, 15)']
Yo BROOO WASSUP <class 'lux.game_map.Cell'>
Super secret. Turn: 0 Action list: ['r 11 5', 'm u_2 s', 'dl 11 5 11 6']
<cla

Yo BROOO WASSUP <class 'lux.game_map.Cell'>
Well.. Shit. I'm unit, u_2 Apparently I can't find any resources. Or it's nighttime
Well.. Shit. I'm unit, u_3 Apparently I can't find any resources. Or it's nighttime
Well.. Shit. I'm unit, u_4 Apparently I can't find any resources. Or it's nighttime
Super secret. Turn: 31 Action list: []
<class 'lux.game_map.Position'>
Closest cluster id is: 9 at position (4, 6) at turn 32
Yo BROOO WASSUP <class 'lux.game_map.Cell'>
Well.. Shit. I'm unit, u_2 Apparently I can't find any resources. Or it's nighttime
Well.. Shit. I'm unit, u_3 Apparently I can't find any resources. Or it's nighttime
Well.. Shit. I'm unit, u_4 Apparently I can't find any resources. Or it's nighttime
Super secret. Turn: 32 Action list: []
<class 'lux.game_map.Position'>
Closest cluster id is: 9 at position (4, 6) at turn 33
Yo BROOO WASSUP <class 'lux.game_map.Cell'>
Well.. Shit. I'm unit, u_2 Apparently I can't find any resources. Or it's nighttime
Well.. Shit. I'm unit, u_3 A

Yo BROOO WASSUP <class 'lux.game_map.Cell'>
Super secret. Turn: 68 Action list: ['m u_2 c', 'dl 13 8 13 8', 'dx 13 8', 'dx 13 8', 'm u_5 e', 'dl 12 7 13 8', 'dx 13 7', 'dx 13 7', 'dx 13 8']
[WARN] (match_VE4DVAOZgk61) - turn 68; Unit u_5 collided when trying to move e to (13, 7)
<class 'lux.game_map.Position'>
Closest cluster id is: 9 at position (3, 6) at turn 69
Yo BROOO WASSUP <class 'lux.game_map.Cell'>
Super secret. Turn: 69 Action list: ['r 12 5', 'm u_2 n', 'dl 13 8 13 7', 'm u_3 s', 'dl 13 7 13 8', 'dx 13 8', 'dx 13 7', 'dx 13 7', 'dx 13 7', 'dx 13 8', 'm u_9 e', 'dl 12 6 13 8']
<class 'lux.game_map.Position'>
Closest cluster id is: 9 at position (3, 6) at turn 70
Yo BROOO WASSUP <class 'lux.game_map.Cell'>
Super secret. Turn: 70 Action list: ['r 11 5', 'm u_4 e', 'dl 12 8 13 8', 'm u_5 e', 'dl 12 7 13 8', 'dx 13 7', 'dx 13 7', 'dx 13 8']
[WARN] (match_VE4DVAOZgk61) - turn 70; Unit u_4 collided when trying to move e to (13, 8)
[WARN] (match_VE4DVAOZgk61) - turn 70; Unit u_5 col

Yo BROOO WASSUP <class 'lux.game_map.Cell'>
Super secret. Turn: 101 Action list: ['r 10 10', 'm u_2 e', 'dl 13 10 14 10', 'dx 14 10', 'dx 14 10', 'dx 14 10', 'dx 14 10', 'dx 14 10', 'dx 14 10', 'm u_12 n', 'dl 13 1 13 0', 'dx 14 10', 'm u_16 n', 'dl 12 11 14 10']
[WARN] (match_VE4DVAOZgk61) - turn 101; Unit u_2 collided when trying to move e to (14, 10)
[WARN] (match_VE4DVAOZgk61) - turn 101; Unit u_16 collided when trying to move n to (12, 10)
<class 'lux.game_map.Position'>
Closest cluster id is: 9 at position (3, 6) at turn 102
Yo BROOO WASSUP <class 'lux.game_map.Cell'>
Super secret. Turn: 102 Action list: ['m u_2 e', 'dl 13 10 14 10', 'dx 14 10', 'dx 14 10', 'dx 14 10', 'dx 14 10', 'dx 14 10', 'dx 14 10', 'm u_10 e', 'dl 12 10 14 10', 'm u_11 e', 'dl 13 9 14 10', 'm u_13 e', 'dl 11 10 14 10', 'dx 14 10', 'm u_15 s', 'dl 14 8 14 10', 'dx 12 10']
[WARN] (match_VE4DVAOZgk61) - turn 102; Unit u_2 collided when trying to move e to (14, 10)
<class 'lux.game_map.Position'>
Closest cluste

In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
# Press shift+O (toggle outputs) to let the output be the whole size
# Display the graphic of the game 
env.render(mode="ipython", width=1200, height=800)

# Seed Compilation
"seed": 826996343 Small map, agents start right next to each other and on the corner of a map.
451898847 Big map, *very* sparse in terms of tree density/space. There's a bunch of uranium though

## File writing / submission 

Basically, just copy paste all your helper functions + agent down here when you're done and then make a tar zip thingy.

To make sure our agent actually works when we submit, copy & paste the agent below and run it for a game (our validation). Because the %%writefile agent.py means that it literally just writes to the file and doesn't bother executing the code into this notebook.
**We moved our submission to submission notebook**

## Create a submission
Now we need to create a .tar.gz file with main.py (and agent.py) at the top level. We can then upload this!

## Submit
Now open the /kaggle/working folder and find submission.tar.gz, download that file, navigate to the "MySubmissions" tab in https://www.kaggle.com/c/lux-ai-2021/ and upload your submission! It should play a validation match against itself and once it succeeds it will be automatically matched against other players' submissions. Newer submissions will be prioritized for games over older ones. Your team is limited in the number of succesful submissions per day so we highly recommend testing your bot locally before submitting.

## CLI Tool

There's a separate CLI tool that can also be used to run matches. It's recommended for Jupyter Notebook users to stick with just this notebook, and all other users including python users to follow the instructions on https://github.com/Lux-AI-Challenge/Lux-Design-2021

The other benefit however of using the CLI tool is that it generates much smaller, "stateless" replays and also lets you run a mini leaderboard on multiple bots ranked by various ranking algorithms

## Additional things to check out

Make sure you check out the Bot API at https://github.com/Lux-AI-Challenge/Lux-Design-2021/tree/master/kits

This documents what you can do using the starter kit files in addition to telling you how to use the annotation debug commands that let you annotate directly on a replay (draw lines, circle things etc.)

You can also run the following below to save a episode to a JSON replay file. These are the same as what is shown on the leaderbaord and you can upload the replay files to the online replay viewer https://2021vis.lux-ai.org/


For a local (faster) version of the replay viewer, follow installation instructions here https://github.com/Lux-AI-Challenge/Lux-Viewer-2021

In [13]:
import json
replay = env.toJSON()
with open("replay.json", "w") as f:
    json.dump(replay, f)

In [22]:
import numpy as np
# https://stackoverflow.com/questions/34325879/how-to-efficiently-find-clusters-of-like-elements-in-a-multidimensional-array
# https://stackoverflow.com/questions/33338202/filling-matrix-with-array-of-coordinates-in-python

# To get the proper array below, we do
# X = [.] # x positions of all trees
# Y = [...] # y positions of all trees
# Z = 1 
# matrix = np.zeros([game_state.width,game_state.height])
# matrix[X,Y] = Z # Sets the tree positions to '1'.

# Wait....

# def researched(player, resource):
#     """
#     given a Resource object, return whether the player has researched the resource type
#     """
#     if resource.type == Constants.RESOURCE_TYPES.WOOD:
#         return True
#     if resource.type == Constants.RESOURCE_TYPES.COAL \
#         and player.research_points >= GAME_CONSTANTS['PARAMETERS']['RESEARCH_REQUIREMENTS']['COAL']:
#             return True
#     if resource.type == Constants.RESOURCE_TYPES.URANIUM \
#         and player.research_points >= GAME_CONSTANTS['PARAMETERS']['RESEARCH_REQUIREMENTS']['URANIUM']:
#             return True
#     return False

def create_tree_matrix(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    matrix = np.zeros([width, height])
    
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            try:
                print(cell.resource.type)
            except:
                pass
            if cell.has_resource() and cell.resource.type == Constants.RESOURCE_TYPES.WOOD: # If it's a tree add it to the matrix.
                matrix[y][x] = 1
#             if cell.has_resource() and cell.resource.type == Constants.RESOURCE_TYPES.COAL: # If it's a coal add it to the matrix.
#                 matrix[y][x] = 2
#             if cell.has_resource() and cell.resource.type == Constants.RESOURCE_TYPES.URANIUM: # If it's a coal add it to the matrix.
#                 matrix[y][x] = 3
    return matrix

# print(game_state.map.get_cell(7,6))
tree_matrix = create_tree_matrix(game_state)
print(tree_matrix)

wood
uranium
uranium
wood
wood
wood
wood
wood
wood
wood
wood
wood
wood
wood
wood
wood
wood
wood
wood
wood
coal
coal
coal
coal
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [23]:
# from pylab import *
from scipy.ndimage import measurements

tree_clusters, num_clusters = measurements.label(tree_matrix)
print("Unique tree clusters: \n", tree_clusters) # Prints list of unique clusters (you can see that the highest number here is the number of clusters in the map).
print(num_clusters)


area = measurements.sum(tree_matrix, tree_clusters, index=arange(tree_clusters.max() + 1))
print(sum(area)) # Total number of trees on map


# Then, for each agent, we want to find the closest non_zero cluster THAT HASN'T BEEN PICKED YET. HOLY SHIT THAT'S IT. THIS IS THE MOST IMPORTANT FUCKING LINE IN THIS WHOLE NTOEBOOK

non_zero_positions = tree_clusters.nonzero()
print("A non_zero tree cluster positions",non_zero_positions, non_zero_positions[0], non_zero_positions[1]) # https://numpy.org/doc/stable/reference/generated/numpy.nonzero.html
# thing[0] is y, thing[1] is x. (a 16x16 map goes 0-15)

def manhattan_distance(x1,x2,y1,y2):
    return abs(x1-x2) + abs(y1-y2)

def closest_cluster(non_zero_positions, matrix):
    """Returns the number of the closest non-zero cluster and its position as a TUPLE
    so
    Args:
    Unit
    non_zero_pos - A list of 2 arrays [[y], [x]] coordinates of the positions of clusters 
    matrix - Matrix of unique cluster locations
    Returns: [11, (10, 9)] or some shit -> cluster_id 11 at position (10, 9)"""
#     start = unit.pos
    
    y_list = non_zero_positions[0] # A list
    x_list= non_zero_positions[1] # A list
    
    closest_dist = math.inf
    for i in range(len(y_list)): # Both x,y same length:
        y= y_list[i]
        x = x_list[i]
    
#         dist = manhattan_distance(start.x, x, start.y, y)
        dist = manhattan_distance(10, x, 2, y) # test
        
        if dist < closest_dist:
            closest_dist = dist
            closest_cluster_num = matrix[y][x]
            # So we can return it later for other functions
            bestx = x
            besty = y 
            
    return [closest_cluster_num, (bestx,besty)] 

print(closest_cluster(non_zero_positions, tree_clusters)) # HOLY FK IT WORKS
tree_clusters[2][10] = 500 # Unit place. # [y][x]
print(tree_clusters)
#closest_cluster = closest_cluster(unit, non_zero_tree_cluster)
# Now we just go to it (if another agent hasn't called dibs on it already).
def move_to_cluster(unit, closest_cluster):
    """
    Args:
    Unit - Unit object
    closest_cluster: List of cluster ID and closest tuple to a worker, as [11, (10,9)] example.
    
    Returns: NA, moves unit to fkin closest resource in that cluster ID"""
    cluster_id, cluster_resource_pos = closest_cluster
    move_unit(unit, cluster_resource_pos) # Might have to change move unit so we can accept tuple inputs / change cluster-pos to a Pos object.
    
    # We still have to do our 'check if agent already has dibs on it' thing but yeah.

Unique tree clusters: 
 [[ 0  1  0  0  0  0  0  0  0  0  0  0  0  0  2  0]
 [ 3  0  4  0  0  0  0  0  0  0  0  0  0  5  0  6]
 [ 0  7  0  0  0  0  0  0  0  0  0  0  0  0  8  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  9  0 10  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 11 11 11  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 11 11  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0 11 11  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
12
18.0
A non_zero tree cluster positions (array([ 0,  0,  1,  1,  1,  1,  2,  2,  6,  6,  8,  8,  8,  9,  9, 10, 10,
       10], dtype=int64), array([ 1, 14, 

In [16]:

a = 0
b = 1


# Ah, it fuckin considers 0's as their own 'island'. We only care about trees.
# def find_clusters(array):
#     clustered = np.empty_like(array)
#     unique_vals = np.unique(array)
#     cluster_count = 0
#     for val in unique_vals:
#         labelling, label_count = ndimage.label(array == val)
#         for k in range(1, label_count + 1):
#             clustered[labelling == k] = cluster_count
#             cluster_count += 1
#     return clustered, cluster_count

# clusters, cluster_count = find_clusters(tree_matrix)
# print("Found {} clusters:".format(cluster_count))
# print(clusters)

# ones = np.ones_like(tree_matrix, dtype=int)
# # print(ones)

# cluster_sizes = ndimage.sum(ones, labels=clusters, index=range(cluster_count)).astype(int)
# com = ndimage.center_of_mass(ones, labels=clusters, index=range(cluster_count))
# for i, (size, center) in enumerate(zip(cluster_sizes, com)):
#     print("Cluster #{}: {} elements at {}".format(i, size, center))

In [27]:
# https://stackoverflow.com/questions/34325879/how-to-efficiently-find-clusters-of-like-elements-in-a-multidimensional-array
# https://stackoverflow.com/questions/33338202/filling-matrix-with-array-of-coordinates-in-python

In [45]:
# Trying new method
# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

def is_tree(cell):
    return cell.has_resource() and cell.resource.type == Constants.RESOURCE_TYPES.WOOD
    

def find_clusters(game_state):
    """Returns {'1' (cluster1): [Tree1, Tree2, ...], '2' (clsuter2): [Tree1, Tree2, ...], ...}"""
    
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            while is_tree(cell):
                # Go check right, down to see if they're also trees (because we come from the top, left)..
                cell_right = game_state.map.get_cell(x+1, y)
                cell_down = game_state.map.get_cell(x, y+1)
                
                if is_tree(cell_right) and is_tree(cell_down):
                    pass:
                elif is_tree(cell_right):
                    
                
                
                
                
                
                
                
                
                resource_tiles.append(cell)
    return resource_tiles

IndentationError: expected an indented block (<ipython-input-45-40c9adf61868>, line 38)